In [1]:
import numpy as np
import scipy.sparse as sp
import math
import pandas as pd

In [2]:
l09 = - math.log10(0.9)
l05 = - math.log10(0.5)

core_cube_gat2016 = np.zeros((3, 3, 6))
# werewolf might not vote possessed
core_cube_gat2016[1, 2, 0] = l05
# possessed might not vote werewolf
core_cube_gat2016[2, 1, 0] = l09
# werewolf would not vote werewolf
core_cube_gat2016[1, 1, 0] = 1.0
# possessed might not divine werewolf as werewolf
core_cube_gat2016[2, 1, 2] = l09
# werewolf might not divine possessed as werewolf
core_cube_gat2016[1, 2, 2] = l05
# werewolf would not divine werewolf as werewolf
core_cube_gat2016[1, 1, 2] = 1.0
# village Seer should not tell a lie
core_cube_gat2016[0, 0, 2] = 2.0
core_cube_gat2016[0, 2, 2] = 2.0
core_cube_gat2016[0, 1, 1] = 2.0
# village Medium should not tell a lie
core_cube_gat2016[0, 0, 4] = 2.0
core_cube_gat2016[0, 2, 4] = 2.0
core_cube_gat2016[0, 1, 3] = 2.0
# village Bodyguard should not tell a lie
# core_cube_gat2016[0, 1, 5] = 2.0

core_cube_gat2016 = -np.log(10)*core_cube_gat2016

In [3]:
class EnemyCaseTensor(object):
    """
    X_3d
    [i, j, 0] : agent i voted agent j (not in talk, action)
    [i, j, 1] : agent i divined agent j HUMAN
    [i, j, 2] : agent i divined agent j WEREWOLF
    [i, j, 3] : agent i inquested agent j HUMAN
    [i, j, 4] : agent i inquested agent j WEREWOLF
    [i, j, 5] : agent i managed to guard agent j
    X_2d
    [i, 0] : agent i is executed
    [i, 1] : agent i is attacked
    [i, 2] : agent i comingout himself/herself SEER
    [i, 3] : agent i comingout himself/herself MEDIUM
    [i, 4] : agent i comingout himself/herself BODYGUARD
    [i, 5] : agent i comingout himself/herself VILLAGER
    [i, 6] : agent i comingout himself/herself POSSESSED
    [i, 7] : agent i comingout himself/herself WEREWOLF
    """
    def __init__(self, core_cube_base, vilsize=15, n_wolves=3, n_features_3d=6, n_features_2d=8):
        # input parameters
        self.vilsize = vilsize
        self.n_wolves = n_wolves
        if self.n_wolves == 3:
            self.n_cases = vilsize * (vilsize - 1) * (vilsize - 2) * (vilsize - 3) / 6
        elif self.n_wolves == 2:
            self.n_cases = vilsize * (vilsize - 1) * (vilsize - 2) / 2
        else:
            raise ValueError('n_wolves must be 3 or 2')
        self.n_features_3d = n_features_3d
        self.n_features_2d = n_features_2d
        # info for the game
        self.X_3d = np.zeros((self.vilsize, self.vilsize, self.n_features_3d))
        self.X_2d = np.zeros((self.vilsize, self.n_features_2d))
        self.F_2d = np.zeros((self.n_cases, 3, self.n_features_2d))
        self.alive = np.zeros((self.n_cases, 3))
        # prob for cases
        self.Y_3d = np.zeros(self.n_cases)
        self.Y_2d = np.zeros(self.n_cases)
        self.Z_2d = np.ones(self.n_cases)
        # linear maps represent v/w/p for each cases, fixed, heavy to initailize
        self.T_2d = np.zeros((self.n_cases, 3, self.vilsize), dtype="int")
        self.T_4d = np.zeros((self.n_cases, 3, 3, self.vilsize, self.vilsize), dtype="int")
        # model parameter cube for X_3d in shape (3, 3, self.vilsize, self.vilsize, self.n_features_3d)
        self.core_cube = np.tensordot(
            np.ones((self.vilsize, self.vilsize, 1)), 
            core_cube_base.reshape((1, 3, 3, self.n_features_3d)), 
            axes=(2, 0)
        ).transpose((2, 3, 0, 1, 4))
        
    def initialize_X(self):
        self.X_3d = np.zeros((self.vilsize, self.vilsize, self.n_features_3d))
        self.X_2d = np.zeros((self.vilsize, self.n_features_2d))
    
    def initialize_T_2d(self):
        # make Tensor
        self.T_2d[:, 0, :] = 1
        if self.n_wolves == 3:
            ind = 0
            for w1 in range(self.vilsize - 2):
                for w2 in range(w1 + 1, self.vilsize - 1):
                    for w3 in range(w2 + 1, self.vilsize):
                        for p in range(self.vilsize):
                            if p not in [w1, w2, w3]:
                                self.T_2d[ind, 0, w1] = 0
                                self.T_2d[ind, 1, w1] = 1
                                self.T_2d[ind, 0, w2] = 0
                                self.T_2d[ind, 1, w2] = 1
                                self.T_2d[ind, 0, w3] = 0
                                self.T_2d[ind, 1, w3] = 1
                                self.T_2d[ind, 0, p] = 0
                                self.T_2d[ind, 2, p] = 1
                                ind += 1
        elif self.n_wolves == 2:
            ind = 0
            for w1 in range(self.vilsize - 2):
                for w2 in range(w1 + 1, self.vilsize - 1):
                    for p in range(self.vilsize):
                        if p not in [w1, w2]:
                            self.T_2d[ind, 0, w1] = 0
                            self.T_2d[ind, 1, w1] = 1
                            self.T_2d[ind, 0, w2] = 0
                            self.T_2d[ind, 1, w2] = 1
                            self.T_2d[ind, 0, p] = 0
                            self.T_2d[ind, 2, p] = 1
                            ind += 1
        else:
            pass
        
    def initialize_T_4d(self):    
        for ind in range(self.n_cases):
            self.T_4d[ind, :, :, :, :] = np.tensordot(
                self.T_2d[ind, :, :].reshape((3, self.vilsize, 1)), 
                self.T_2d[ind, :, :].reshape((1, 3, self.vilsize)), 
                axes=(2, 0)
            ).transpose((0, 2, 1, 3))
        
    def update_Y(self):
        self.Y_3d = np.tensordot(
            self.T_4d, self.core_cube*np.tensordot(
                np.ones((3, 3, 1)), 
                self.X_3d.reshape((1, self.vilsize, self.vilsize, self.n_features_3d)), axes=(2, 0)
            ), axes=([1, 2, 3, 4], [0, 1, 2, 3])
        ).sum(axis=1)
        self.F_2d = np.tensordot(
            self.T_2d, self.X_2d, axes=(2, 0)
        )
        self.Y_2d, self.Z_2d = self.fit_co_alive(self.F_2d)
        
    def fit_co_alive(self, F_2d):
        Y_co = np.zeros((F_2d.shape[0]))
        self.alive[:, 0] = self.vilsize - self.n_wolves - 1
        self.alive[:, 1] = self.n_wolves
        self.alive[:, 2] = 1
        self.alive -= (F_2d[:, :, 0] + F_2d[:, :, 1])
        Z_alive = np.ones((F_2d.shape[0]))
        for i in range(F_2d.shape[0]):
            # village Seer would comingout
            if F_2d[i, 2, 2] + F_2d[i, 1, 2] > 0 and F_2d[i, 0, 2] == 0:
                Y_co[i] += 2.0
            # village Medium would comingout
            if F_2d[i, 2, 3] + F_2d[i, 1, 3] > 0 and F_2d[i, 0, 3] == 0:
                Y_co[i] += 2.0
            # village Bodyguard would comingout
            if F_2d[i, 2, 4] + F_2d[i, 1, 4] > 0 and F_2d[i, 0, 4] == 0:
                Y_co[i] += 2.0
            # possessed would comingout
            if F_2d[i, 2, 2] + F_2d[i, 2, 3] + F_2d[i, 2, 4] == 0:
                Y_co[i] += 1.0
            # werewolves might not comingout
            Y_co[i] += (F_2d[i, 2, 2] + F_2d[i, 2, 3] + F_2d[i, 2, 4]) * 0
            # villagers must not comingout
            if F_2d[i, 0, 2] > 1:
                Y_co[i] += 3.0
            if F_2d[i, 0, 3] > 1:
                Y_co[i] += 3.0
            if F_2d[i, 0, 4] > 1:
                Y_co[i] += 3.0
            # werewolves are never to be attacked
            if F_2d[i, 1, 1] > 0:
                Z_alive[i] *= 0
            # game is going on, there's at least one werewolf
            if self.alive[i, 1] <= 0:
                Z_alive[i] *= 0
            # game is going on, humans > wolves
            if self.alive[i, 1] >= self.alive[i, 0] + self.alive[i, 2]:
                Z_alive[i] *= 0
        return -np.log(10)*Y_co, Z_alive
    
    # to return 1
    def prob_now(self):
        """return [P(agent_idx = VILLAGER), P(agent_idx = POSSESSED), P(agent_idx = WEREWOLF)]
        """
        return np.tensordot(self.T_2d, 
                            self.Z_2d*np.exp(self.Y_3d + self.Y_2d) / np.sum(self.Z_2d*np.exp(self.Y_3d + self.Y_2d)), 
                            axes=[0, 0]).transpose((1,0))
    
    
    ####################
    # methods for X_3d #
    ####################
    def get_votelist(self, votelist):
        """get votelist as list of list [[x, y],...]
        x:agent, y:target
        """
        # [i, j, 0] : agent i voted agent j (not in talk, action)
        for vote in votelist:
            self.X_3d[vote[0], vote[1], 0] += 1
            
    def get_divine(self, divine):
        """get a new divine result in form [x, y, race]
        x:agent, y:target, race:0:"HUMAN", 1:"WEREWOLF"
        """
        # [i, j, 1] : agent i divined agent j HUMAN
        # [i, j, 2] : agent i divined agent j WEREWOLF
        self.X_3d[divine[0], divine[1], divine[2] + 1] += 1
    
    def get_inquest(self, inquest):
        """get a new inquest result in form [x, y, race]
        x:agent, y:target, race:0:"HUMAN", 1:"WEREWOLF"
        """
        # [i, j, 3] : agent i inquested agent j HUMAN
        # [i, j, 4] : agent i inquested agent j WEREWOLF
        self.X_3d[inquest[0], inquest[1], inquest[2] + 3] += 1
    
    def get_guard(self, guard):
        """get a new guard result in form [x, y]
        x:agent, y:target
        """
        # [i, j, 5] : agent i guarded agent j
        self.X_3d[guard[0], guard[1], 5] += 1
    
    ####################
    # methods for X_2d #
    ####################
    def get_executed(self, x):
        """get agentid newly executed
        x:agent
        """
        # [i, 0] : agent i is executed
        self.X_2d[x, 0] = 1
            
    def get_attacked(self, x):
        """get agentid newly attacked by werewolves
        x:agent
        """
        # [i, 1] : agent i is attacked
        self.X_2d[x, 1] = 1
            
    def get_co(self, co):
        """get a new co in form [x, role]
        x:agent, role:0:"SEER", 1:"MEDIUM", 2:"BODYGUARD", 3:"VILLAGER", 4:"POSSESSED", 5:"WEREWOLF"
        """
        # [i, 2] : agent i comingout himself/herself SEER
        # [i, 3] : agent i comingout himself/herself MEDIUM
        # [i, 4] : agent i comingout himself/herself BODYGUARD
        # [i, 5] : agent i comingout himself/herself VILLAGER
        # [i, 6] : agent i comingout himself/herself POSSESSED
        # [i, 7] : agent i comingout himself/herself WEREWOLF
        self.X_2d[co[0], co[1] + 2] += 1
    
        


In [4]:
# もろもろ継承略
# 0:メイソン
# 1:ダンカン
# 2:デイジー
# 3:大浜
# 4:武中
# 5:香川
# 6:児玉
# 7:中田
# 8:村中
# 9:安西
# 10:大野
# 11:森本
# 12:イシイ
agent_names = ["0:メイソン", "1:ダンカン", "2:デイジー", "3:大浜", "4:武中", "5:香川", "6:児玉", "7:中田", "8:村中", 
              "9:安西", "10:大野", "11:森本", "12:イシイ"]

In [5]:
game2_1 = EnemyCaseTensor(core_cube_base=core_cube_gat2016, vilsize=13)
game2_1.initialize_T_2d()
game2_1.initialize_T_4d()
game2_1.initialize_X()

In [6]:
np.set_printoptions(suppress=True)
def print_matrix(A, agent_names):
    return pd.DataFrame(A, index=agent_names, columns=["村", "人狼", "狂人"])

In [7]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.692308,0.230769,0.076923
1:ダンカン,0.692308,0.230769,0.076923
2:デイジー,0.692308,0.230769,0.076923
3:大浜,0.692308,0.230769,0.076923
4:武中,0.692308,0.230769,0.076923
5:香川,0.692308,0.230769,0.076923
6:児玉,0.692308,0.230769,0.076923
7:中田,0.692308,0.230769,0.076923
8:村中,0.692308,0.230769,0.076923
9:安西,0.692308,0.230769,0.076923


In [8]:
# 1日目
# 大浜と村中が占いCO
game2_1.get_co([3, 0])
game2_1.get_co([8, 0])
# 占い結果:大浜->森本(白), 村中->安西(白)
game2_1.get_divine([3, 11, 0])
game2_1.get_divine([8,  9, 0])
# 大野が霊媒CO
game2_1.get_co([10, 1])
# 投票
game2_1.get_votelist([[7, 5], [3, 8], [1, 5], [11, 12], [4, 7], [5, 10], [2, 4], [8, 7], [12, 10], [10, 12], [0, 4], [9, 7], [6, 10]])
# 再投票
game2_1.get_votelist([[0, 7], [1, 7], [2, 10], [3, 7], [4, 7], [5, 7], [6, 10], [8, 7], [9, 7], [11, 7], [12, 10]])



In [9]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.651389,0.316984,0.031628
1:ダンカン,0.665808,0.302716,0.031476
2:デイジー,0.621980,0.346440,0.031580
3:大浜,0.520583,0.143482,0.335936
4:武中,0.748311,0.227569,0.024120
5:香川,0.671241,0.301530,0.027229
6:児玉,0.544902,0.424188,0.030910
7:中田,0.935671,0.057712,0.006617
8:村中,0.475588,0.148270,0.376142
9:安西,0.761202,0.201914,0.036884


In [10]:
# 2日目
# 襲撃:なし
# デイジーと児玉が霊媒CO
game2_1.get_co([2, 1])
game2_1.get_co([6, 1])
# 霊媒結果:デイジー->中田(白), 児玉->中田(白), 大野->中田(白)
game2_1.get_inquest([2, 7, 0])
game2_1.get_inquest([6, 7, 0])
game2_1.get_inquest([10, 7, 0])
# 占い結果:大浜->デイジー(白), 村中->児玉(白)
game2_1.get_divine([3, 2, 0])
game2_1.get_divine([8, 6, 0])
# 投票
game2_1.get_votelist([[10, 6], [2, 6], [0, 10], [1, 10], [6, 10], [4, 6], [5, 10], [12, 10], [11, 6], [3, 6], [9, 6], [8, 10]])
# 再投票
game2_1.get_votelist([[0, 10], [1, 10], [2, 6], [3, 6], [4, 6], [5, 10], [8, 6], [9, 10], [11, 6], [12, 6]])



In [11]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.908491,0.088597,0.002911
1:ダンカン,0.909063,0.088026,0.002911
2:デイジー,0.029195,0.378764,0.592041
3:大浜,0.115185,0.847552,0.037263
4:武中,0.715300,0.282820,0.001880
5:香川,0.930292,0.067132,0.002576
6:児玉,0.840914,0.075062,0.084024
7:中田,0.997775,0.000697,0.001528
8:村中,0.884468,0.110859,0.004673
9:安西,0.984934,0.011753,0.003313


In [12]:
# 3日目
# 襲撃:メイソン
game2_1.get_attacked(0)
# 霊媒結果:デイジー->児玉(黒), 大野->児玉(黒)
game2_1.get_inquest([2, 6, 1])
game2_1.get_inquest([10, 6, 1])
# 占い結果:大浜->メイソン(白), 村中->メイソン(白)
game2_1.get_divine([3, 0, 0])
game2_1.get_divine([8, 0, 0])
# 投票
game2_1.get_votelist([[3, 8], [2, 8], [1, 10], [8, 10], [11, 10], [4, 2], [10, 8], [12, 10], [9, 10], [5, 10]])



In [13]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.992642,0.000000,0.007358
1:ダンカン,0.868463,0.124923,0.006614
2:デイジー,0.001315,0.402603,0.596083
3:大浜,0.147130,0.841786,0.011084
4:武中,0.622873,0.373035,0.004092
5:香川,0.898384,0.095666,0.005950
6:児玉,0.851198,0.146988,0.001814
7:中田,0.993376,0.002094,0.004531
8:村中,0.852824,0.146599,0.000577
9:安西,0.984515,0.008130,0.007355


In [14]:
# 4日目
# 襲撃:イシイ
game2_1.get_attacked(12)
# 霊媒結果:デイジー->大野(黒)
game2_1.get_inquest([2, 10, 1])
# 占い結果:大浜->ダンカン(白), 村中->森本(白)
game2_1.get_divine([3, 1, 0])
game2_1.get_divine([8, 11, 0])
# 投票
game2_1.get_votelist([[5, 2], [1, 2], [4, 2], [3, 4], [2, 10], [9, 8], [8, 2], [11, 2]])

In [15]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.983694,0.000000,0.016306
1:ダンカン,0.909810,0.076982,0.013207
2:デイジー,0.000598,0.214939,0.784463
3:大浜,0.381335,0.616266,0.002400
4:武中,0.624759,0.371164,0.004077
5:香川,0.586785,0.401331,0.011885
6:児玉,0.615220,0.381628,0.003152
7:中田,0.966664,0.023274,0.010062
8:村中,0.618568,0.381177,0.000254
9:安西,0.961995,0.021700,0.016306


In [16]:
# 5日目
# 襲撃:森本
game2_1.get_attacked(11)
# 占い結果:大浜->武中(白), 村中->武中(黒)
game2_1.get_divine([3, 4, 0])
game2_1.get_divine([8, 4, 1])
# 投票
game2_1.get_votelist([[4, 8], [8, 4], [3, 8], [1, 4], [5, 4], [9, 4]])



In [17]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.999777,0.000000,0.000223
1:ダンカン,0.993936,0.005883,0.000181
2:デイジー,0.000704,0.003992,0.995304
3:大浜,0.505135,0.494592,0.000274
4:武中,0.508353,0.491592,0.000055
5:香川,0.509141,0.490698,0.000161
6:児玉,0.494214,0.505136,0.000649
7:中田,0.999435,0.000430,0.000135
8:村中,0.494800,0.504936,0.000264
9:安西,0.989694,0.010084,0.000222


In [18]:
# 6日目
# 襲撃:ダンカン
game2_1.get_attacked(1)
# 占い結果:大浜->香川(白), 村中->大浜(黒)
game2_1.get_divine([3, 5, 0])
game2_1.get_divine([8, 3, 1])

In [19]:
game2_1.update_Y()
print_matrix(game2_1.prob_now(), agent_names)

,村,人狼,狂人
0:メイソン,0.999567,0.000000,0.000433
1:ダンカン,0.999649,0.000000,0.000351
2:デイジー,0.001249,0.005405,0.993346
3:大浜,0.029297,0.970696,0.000007
4:武中,0.033633,0.966260,0.000107
5:香川,0.989140,0.010544,0.000316
6:児玉,0.969475,0.029282,0.001243
7:中田,0.998937,0.000797,0.000265
8:村中,0.970590,0.029143,0.000267
9:安西,0.979774,0.019793,0.000433
